In [34]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%------------------------------------------------------%
#%
#% Machine Perception and Cognitive Robotics Laboratory
#%
#%     Center for Complex Systems and Brain Sciences
#%               
#%              Florida Atlantic University
#%
#%------------------------------------------------------%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%------------------------------------------------------%
#% William Hahn & Elan Barenholtz
#% Feedback Amplifier 
#% Backpropagation Neural Network
#% Pedestrian Detection
#% Vectorized Backpropagation Neural Network Demonstration 
#% See:
#% Daimler Pedestrian Classification Benchmark Dataset
#% S. Munder and D. M. Gavrila. 
#% An Experimental Study on Pedestrian Classification. 
#% IEEE Transactions on Pattern Analysis and Machine Intelligence, 
#% vol. 28, no. 11, pp.1863-1868, November 2006.
#%------------------------------------------------------%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%------------------------------------------------------%
#function MPCR_NN_Pedestrian_FA

from pylab import *
import numpy as np
import scipy as sci
import scipy.io as sio
from scipy.io import loadmat
import matplotlib
import random
from numpy import dot as matrixmultiply


data = loadmat('Daimler_ped_data.mat');
c1=data['c1']
c2=data['c2']

def im2double(im):
    min_val = np.min(im.ravel())
    max_val = np.max(im.ravel())
    out = (im.astype('float') - min_val) / (max_val - min_val)
    return out

def af(weighted_sum):
    af = 1/(1+np.exp(-weighted_sum));  		# Logistic / Sigmoid Function
    return af

def network_output (neural_input,hidden_weights,output_weights):
    niconc = np.concatenate(neural_input,1);
    niconchw = matrixmultiply(niconc,hidden_weights);
    network_output = af(matrixmultiply(concatenate(af(niconchw),1),output_weights));
    return network_output


c1=c1.reshape(648,5000)
c2=c2.reshape(648,4800)
pattern=np.concatenate((im2double(c1), im2double(c2)),axis=1)
pattern=pattern.transpose();

categoryc1=ones((1,c1.shape[1]));
categoryc2=0.*ones((1,c2.shape[1]));
category=np.concatenate((categoryc1, categoryc2), axis=1);
category=category.transpose();

r=np.random.permutation(size(pattern,0));

pattern=pattern[r,];
category=category[r,];

trainpattern=pattern[:pattern.shape[0]/3,];

testpattern=pattern[1+(pattern.shape[0]/3):2*(pattern.shape[0]/3),];

validpattern=pattern[1+(2*(pattern.shape[0]/3)):pattern.shape[0],];

traincategory=category[:pattern.shape[0]/3,];

testcategory=category[1+(pattern.shape[0]/3):2*(pattern.shape[0]/3),];

validcategory=category[1+(2*(pattern.shape[0]/3)):pattern.shape[0],];

#for i in range (1,pattern.shape[1]):
#
#    plot.imshow(arrange(pattern[i-1,]).reshape(36,18))
#    colormap(gray)
#
#    category(i)
#
#    pause
#
#end

biastrain=ones((size(trainpattern,0),1));
trainpattern = np.concatenate((trainpattern, biastrain),axis=1);
biastest=ones((size(testpattern,0),1));
testpattern = np.concatenate((testpattern, biastest),axis=1);
biasvalid=ones((size(validpattern,0),1));
validpattern = np.concatenate((validpattern, biasvalid),axis=1);


n1 = size(trainpattern,axis=1)
n2 = 2;   #%n2-1
n3 = size(traincategory,axis=1);

w1 = 0.001*(1-2*np.random.rand(n1,n2-1));
w2 = 0.001*(1-2*np.random.rand(n2,n3));

dw1 = zeros(w1.shape);
dw2 = zeros(w2.shape);

L = 0.0001;        #% Learning
M = 0.8;          #% Momentum




#p=0.01;

errorplot=[];

for loop in range(1,5000):

    b=.1;
    
    act1 = matrixmultiply(b,trainpattern);
    act1 = matrixmultiply(act1,w1);
    act1 = np.concatenate((act1, biastrain),axis=1);
    act1 = af(act1)
    
    #act1 = np.concatenate((af(matrixmultiply((matrixmultiply(b,trainpattern)),w1)), bias),axis=1);
    act2 = matrixmultiply(act1,w2);
    act2 = af(act2);
    
    error = traincategory - act2;
    sse = sum(error**2); 
    
    delta_w2 = error * act2 * (1-act2);
    delta_w1 = matrixmultiply(delta_w2,(w2.transpose()));
    delta_w1 = np.multiply(delta_w1,act1);
    delta_w1 = np.multiply(delta_w1,(1-act1));

    delta_w1=sci.delete(delta_w1,size(delta_w1,axis=1)-1,1);

    dw1 = matrixmultiply((L * trainpattern.transpose()),delta_w1) + M * dw1; 
    dw2 = matrixmultiply((L * act1.transpose()),delta_w2) + M * dw2;
    
    w1 = w1 + dw1;
    w2 = w2 + dw2;
    
  
    figure(1)
    subplot(121)
    #imagesc(np.arrange(w1[1:n1,0]).reshape(36,18))
    subplot(122)
    subplot(174)
    
    nn=range(1,size(testcategory,axis=1)+1);
    plot(nn[np.argwhere(traincategory==1)],act2[np.argwhere(traincategory==1)],'g^','MarkerSize',12)
    plot(nn(np.argwhere(traincategory==0)),act2(np.argwhere(traincategory==0)),'rv','MarkerSize',12)
    show()
    plot(traincategory,'bo')
    # still need to do this: "hold off"
    #
    
    subplot(175)
    act1 = np.concatenate(af(matrixmultiply(testpattern,w1)),bias,axis=1);
    #
    act2 = af(matrixmultiply(act1,w2));
    
    #
    nn=range(1,size(testcategory,axis=1)+1);
    plot(nn(find(testcategory==1)),act2(find(testcategory==1)),'g^','MarkerSize',12)
    plot(nn(find(testcategory==0)),act2(find(testcategory==0)),'rv','MarkerSize',12)
    show()
    plot(testcategory,'bo')
    # still need to do this: hold off
    
    error2 = testcategory != (act2>0.5);
    sse2 = sum(error2);
    
    
    subplot(176)
    act1 = np.concatenate(af(matrixmultiply(validpattern,w1),bias,axis=1));
    #%
    act2 = af(matrixmultiply(act1,w2));
   
    #%
    nn=np.range(1,size(testcategory,axis=1)+1);
    plot(nn(find(validcategory==1)),act2(find(validcategory==1)),'g^','MarkerSize',12)
    plot(nn(find(validcategory==0)),act2(find(validcategory==0)),'rv','MarkerSize',12)
    show()
    plot(validcategory,'bo')
    show()
    # still need to do this: hold off
    
    error3 = validcategory != (act2>0.5);
    sse3 = sum(error3);


    ep1=np.concatenate(loop, b, (1-(sse/length(testcategory)))*100, (1-(sse2/length(testcategory)))*100, (1-(sse3/length(validcategory)))*100,axis=1)
    
    errorplot=np.concatenate((errorplot, ep1), axis=0);
    subplot(177)
    plot(errorplot[:,[2,3,4]])

    set(gcf,'color','w');

    drawnow()

 
end


end






TypeError: only integer arrays with one element can be converted to an index

In [12]:
from pylab import *
import numpy as np
import scipy as sci
import scipy.io as sio
from scipy.io import loadmat
import matplotlib
import random
A = np.matrix('1 2 3; 3 4 5; 6 7 8; 9 9 9')
print A
G=A*A
print G
a = size(A,axis=1)
b = size(A,axis=0)
print a
print b
#print np.ravel(A)
B=sci.delete(A,2,0);
print B
C=sci.delete(A,1,0);
print C
D=sci.delete(A,0,0);
print D
E=np.random.rand(3,2)
print E

[[1 2 3]
 [3 4 5]
 [6 7 8]
 [9 9 9]]


ValueError: shapes (4,3) and (4,3) not aligned: 3 (dim 1) != 4 (dim 0)

In [11]:
from pylab import *
import numpy as np
import scipy as sci
import scipy.io as sio
from scipy.io import loadmat
import matplotlib
import random
from numpy import dot as matrixmultiply
print shape(delta_w2)
print shape(w2)
print shape(act1)
print shape(testcategory)
print shape(validpattern)
print shape(validcategory)
print np.random.random(5)
a = []
print a
b = [1,2,3]
f = [4,5,6]
print f
print b
c = np.concatenate((a, b), axis=0)
print c
d = np.concatenate([b,1])
print d


(3266, 1)
(2, 1)
(3266, 2)
(3265, 1)
(3267, 649)
(3267, 1)
[ 0.5614922   0.35188391  0.83136813  0.84595347  0.80307024]
[]
[4, 5, 6]
[1, 2, 3]
[ 1.  2.  3.]


ValueError: all the input arrays must have same number of dimensions